In [2]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
from nibabel.freesurfer.io import read_annot
from nibabel.freesurfer import read_geometry
import matplotlib.pyplot as plt
from nilearn import plotting, surface

ModuleNotFoundError: No module named 'nibabel'

In [ ]:
# STEP 0: paths and imports
import os
import numpy as np
import pandas as pd
import nibabel as nib
from nibabel.freesurfer.io import read_annot
from nibabel.freesurfer import read_geometry
import matplotlib.pyplot as plt
from nilearn import plotting, surface

# Set your FreeSurfer SUBJECTS_DIR and use fsaverage surfaces
SUBJECTS_DIR = os.environ.get("SUBJECTS_DIR", "/path/to/your/freesurfer/subjects")
fsavg = os.path.join(SUBJECTS_DIR, "fsaverage")

lh_surf = os.path.join(fsavg, "surf", "lh.inflated")
rh_surf = os.path.join(fsavg, "surf", "rh.inflated")
lh_annot = os.path.join(fsavg, "label", "lh.aparc.annot")
rh_annot = os.path.join(fsavg, "label", "rh.aparc.annot")

# STEP 1: load your coefficients
csv_path = "/path/to/ON_arp_smri_coefs.csv"
df = pd.read_csv(csv_path)

# Pick only cortical thickness columns
fs_cols = [c for c in df.columns if c.startswith(("lh_", "rh_")) and c.endswith("_thickness")]

# EITHER: average across rows (default)
coef_series = df[fs_cols].mean(axis=0)
# OR: select a single model/run row (e.g., row 0)
# coef_series = df.loc[0, fs_cols]

# Make a dict: ('lh', 'bankssts') -> coef
def parse_feat(name):
    hemi = name.split("_", 1)[0]
    region = name.split("_", 1)[1].replace("_thickness", "")
    return hemi, region

coef_map = {}
for col, val in coef_series.items():
    hemi, region = parse_feat(col)
    coef_map[(hemi, region)] = float(val)

# STEP 2: read fsaverage surfaces and aparc labels
# Surfaces
lh_coords, lh_faces = read_geometry(lh_surf)
rh_coords, rh_faces = read_geometry(rh_surf)

# Annotations (label id per vertex, and the lookup table with names)
lh_labels, lh_ctab, lh_names = read_annot(lh_annot)
rh_labels, rh_ctab, rh_names = read_annot(rh_annot)

# Convert bytes->str for FS region names (exclude 'unknown')
lh_names = [n.decode("utf-8") if isinstance(n, bytes) else n for n in lh_names]
rh_names = [n.decode("utf-8") if isinstance(n, bytes) else n for n in rh_names]

# STEP 3: build per-vertex maps from parcel coefficients
def build_vertex_map(labels, names, hemi, coef_map):
    # names is an array where index corresponds to label id in 'labels'
    # Create lookup from region name -> coef
    # aparc names are like 'bankssts', 'entorhinal', etc.
    name_to_coef = {n: coef_map.get((hemi, n), 0.0) for n in names}
    # Fill per-vertex array
    stat = np.zeros_like(labels, dtype=float)
    for lab_id, reg_name in enumerate(names):
        stat[labels == lab_id] = name_to_coef.get(reg_name, 0.0)
    return stat

lh_stat = build_vertex_map(lh_labels, lh_names, "lh", coef_map)
rh_stat = build_vertex_map(rh_labels, rh_names, "rh", coef_map)

# STEP 4: plot and save images
# Note: not setting any custom colors/styles; using nilearn defaults.
outdir = "./brain_coef_maps"
os.makedirs(outdir, exist_ok=True)

# Load meshes via nilearn for plotting convenience
lh_mesh = surface.load_surf_mesh(lh_surf)
rh_mesh = surface.load_surf_mesh(rh_surf)

views = [("lateral", "lh", lh_mesh, lh_stat, "lh_lateral.png"),
         ("medial",  "lh", lh_mesh, lh_stat, "lh_medial.png"),
         ("lateral", "rh", rh_mesh, rh_stat, "rh_lateral.png"),
         ("medial",  "rh", rh_mesh, rh_stat, "rh_medial.png")]

for view, hemi, mesh, stat, fname in views:
    display = plotting.plot_surf_stat_map(
        surf_mesh=mesh,
        stat_map=stat,
        bg_map=None,
        view=view,
        colorbar=True,
        title=f"{hemi.upper()} {view} (aparc thickness coefs)",
    )
    display.savefig(os.path.join(outdir, fname), dpi=200, bbox_inches="tight")
    display.close()

print(f"Saved images to: {os.path.abspath(outdir)}")
